In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# **Part 1: Cleaning and preprocessing the data**

## **Preprocessing:**

We are working with two different .csv files:

- "Instrument" **`(df)`** contains the bulk of information we're interested in, whereas 
- "Administration" **`(df2)`** provides additional demographic information on each child - in our case we are only interested in sex. 

**Excerpt of the wordbank "instrument" file:**

In [23]:
df = pd.read_parquet("wordbank_instrument_data.parquet")
df.head()

,downloaded,data_id,item_kind,category,item_id,item_definition,child_id,age,value
index,,,,,,,,,
0,2023-09-03,245518,word,sounds,item_1,baa baa,1,28,produces
1,2023-09-03,245518,word,sounds,item_2,choo choo,1,28,None
2,2023-09-03,245518,word,sounds,item_3,cockadoodledoo,1,28,None
3,2023-09-03,245518,word,sounds,item_4,grrr,1,28,produces
4,2023-09-03,245518,word,sounds,item_5,meow,1,28,None


**Excerpt of the wordbank "administration" file.** 

The completeness of data is not uniform across all datasets types (e.g. `Marchman` vs. `Smith`), making many variables that would be interesting to look at not feasible to include in our analysis because they have a too high count of NAs. We stick to only being interested in the variable **`sex`**. 

In [24]:
df2 = pd.read_parquet("wordbank_administration_data.parquet")
df2.head()

,downloaded,language,form,dataset_name,child_id,age,comprehension,production,is_norming,birth_order,...,race,sex,birth_weight,born_early_or_late,gestational_age,zygosity,language_exposures,health_conditions,monolingual,typically_developing
0,2023-09-03,English (American),WS,Marchman,1,28,497,497,True,Fourth,...,None,Female,NaN,NaN,NaN,NaN,NaN,None,True,True
1,2023-09-03,English (American),WS,Marchman,2,22,369,369,True,Second,...,White,Female,NaN,NaN,NaN,NaN,NaN,None,True,True
2,2023-09-03,English (American),WS,Marchman,3,26,190,190,True,Fourth,...,White,Female,NaN,NaN,NaN,NaN,NaN,None,True,True
3,2023-09-03,English (American),WS,Marchman,4,27,264,264,True,Second,...,White,Male,NaN,NaN,NaN,NaN,NaN,None,True,True
4,2023-09-03,English (American),WS,Marchman,5,19,159,159,True,Second,...,Other,Female,NaN,NaN,NaN,NaN,NaN,None,True,True


Back to our first dataset **(`df`)**. All items that are not of the **`item_kind`** `word` are excluded.

The column **`value`** codes whether the child in question has learned a certain word or not. It has multiple possible values: `produces`, `understands`, `None`. We are only interested in whether a child can speak a word or not, not whether it simply understands the word. `understands` is therefore coded as 0. `None`, meaning a child neither speaks nor understands a given word, is similarly coded as 0.

In [25]:
df = df.loc[df["item_kind"] == "word"]
df["value"].fillna(0, inplace=True)
df["value"].replace("produces", 1, inplace=True)
df["value"].replace("understands", 0, inplace=True)

Let's have a look at our data:

In [26]:
print("\033[1mSummary:\033[0m")
print()
print("Total number of words recorded:", df["item_id"].nunique())
print("Total number of children recorded:", df["child_id"].nunique())
print("Numer of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

Summary:

Total number of words recorded: 680
Total number of children recorded: 6373
Numer of rows: 5168680
Number of columns: 9


Looking at the summary, we should expect to have 680 rows per child.

<font color="red">**HOWEVER:**</font> our df has 5168680 rows, and 6373 x 680 =/= 5168680, so something must be wrong here. Let's investigate, by using the other ID variable, called **`data_id`**.

In [27]:
df["data_id"].nunique()

7601

There are 7601 unique values for **`data_id`**, indicating that this variable does not represent the same information as **`child_id`**, as originally assumed. Also interesting: 7601 x 680 = 5168680, which is in accordance with the n of rows in our df.

Let's investigate the **`child_id`**'s that appear in more than 680 rows. 

In [28]:
#pd.set_option('display.max_rows', None)

grouped = df.groupby("child_id")["downloaded"].count().reset_index()
grouped = grouped[grouped["downloaded"] > 680]
print(grouped)

      child_id  downloaded
1375      1414        1360
1377      1416        2040
1379      1418        2040
1388      1427        2040
1389      1428        1360
...        ...         ...
4198     81430        1360
4199     81431        1360
4200     81432        1360
4201     81433        1360
4202     81434        1360

[818 rows x 2 columns]


- The number of rows tell us that 818 IDs have more entries than expected (>680). 
- The column **`child_id`** indicates which ID has more entries than expected.
- The column **`downloaded`** indicates the number of entries/rows. Notably, the number is always multiples of 680.

Let's pick out one of the child IDs to investigate further.

In [29]:
df[(df['child_id'] == 1414) & (df["item_id"] == "item_1")]

,downloaded,data_id,item_kind,category,item_id,item_definition,child_id,age,value
index,,,,,,,,,
1095875,2023-09-03,246931,word,sounds,item_1,baa baa,1414,16,0
1226583,2023-09-03,247095,word,sounds,item_1,baa baa,1414,18,1


The same pattern shown above can be reproduced for different child IDs and items. A larger value of **`data_id`** is associated with an increase in **`age`**, and in the case of this specific item also a change in **`value`**, indicating that the multiple entries might be due to the same child being recorded at different time points. 

For the above example: the child is two months older at the second time of recording (**`age`**: 16 to 18) and is now able to say `baa baa` (**`value`**: 0 to 1).

In [30]:
df[(df['child_id'] == 2427) & (df["item_id"] == "item_1")]

,downloaded,data_id,item_kind,category,item_id,item_definition,child_id,age,value
index,,,,,,,,,
2379842,2023-09-03,248801,word,sounds,item_1,baa baa,2427,28,1
2383827,2023-09-03,248808,word,sounds,item_1,baa baa,2427,28,1


However, further explorations (example above) reveal that for some IDs, **`age`** does not actually change, and in some instances **`value`** will actually change from 1 to 0, which is kind of strange (unless the child in question really did unlearn the given word). Wordbank unfortunately does not provide any official documentation on the data, making things difficult to interpret definitively.

Let's ignore this matter for now and turn to our second dataset, **`df2`**, in which further demographic characeristics of the children are provided. In a first step, children who are not typically developing have to unfortunately be excluded from our analysis, which stands in stark contrast to the original goal of this application. The reason is that there are too few datapoints for non-typically developing children (n = 234), too many variations in their respective health conditions (3: Autism, Down syndrome, Pre-term), and lastly, too many non-typically developing children for which no information on their health condition is even provided (n = 101).

In [10]:
df2 = df2.loc[df2["typically_developing"] == True]

Check for `NA`s in **`sex`**:

In [31]:
df2["sex"].isna().sum()

1447

Drop them.

In [32]:
df2 = df2.dropna(subset=["sex"])

Now, let's have a look at the data:

In [33]:
print("\033[1mSummary:\033[0m")
print()
print("Total number of children recorded:", df2["child_id"].nunique())
print("Numer of rows:", df2.shape[0])

Summary:

Total number of children recorded: 5242
Numer of rows: 5742


The total number of children, according to **`child_id`** in **`df2`** is not the same as in **`df`** (5242 vs. 6373). Additionally, once again it seems that some IDs have multiple entries. For **`df2`**, each ID should only be represented in 1 row, meaning that we should have 5242 rows, not 5742. The variable **`data_id`** does not exist in **`df2`**.

Let's have a look at one of the IDs with more than 1 entry:

In [34]:
grouped2 = df2.groupby("child_id")["downloaded"].count().reset_index()
grouped2 = grouped2[grouped2["downloaded"] > 1]

In [35]:
df2[df2["child_id"] == 1919]

,downloaded,language,form,dataset_name,child_id,age,comprehension,production,is_norming,birth_order,...,race,sex,birth_weight,born_early_or_late,gestational_age,zygosity,language_exposures,health_conditions,monolingual,typically_developing
2045,2023-09-03,English (American),WS,Marchman,1919,18,21,21,False,Second,...,White,Female,NaN,NaN,NaN,NaN,NaN,None,True,True
2046,2023-09-03,English (American),WS,Marchman,1919,24,138,138,False,Second,...,White,Female,NaN,NaN,NaN,NaN,NaN,None,True,True
2047,2023-09-03,English (American),WS,Marchman,1919,30,511,511,False,Second,...,White,Female,NaN,NaN,NaN,NaN,NaN,None,True,True


Explorations of cases of multiple entries further indicate that the underlying cause here could be that certain children were recorded more than once, at different points in time. **`Birth order`**, **`sex`**, **`race`**, etc. are the same across the "duplicate" entries, whereas the values of **`age`**, **`production`** and **`comprehension`** change (see example above). A possible solution would be to treat the duplicate entries as separate children, since they still contain valuable information. Explore the data a bit more before making a final decision.

At a first glance, it seems that *all* duplicates really are the product of longitudinal data. Note the pattern shown in the table below: **`downloaded`**/**`grouped`** (left side) are always multiples of 680 by a factor of **`downloaded`**/**`grouped2`** (right side).

In [36]:
pd.concat([grouped, grouped2], axis=1)

,child_id,downloaded,child_id,downloaded
1375,1414.0,1360.0,1414.0,2.0
1377,1416.0,2040.0,1416.0,3.0
1379,1418.0,2040.0,1418.0,3.0
1388,1427.0,2040.0,1427.0,3.0
1389,1428.0,1360.0,1428.0,2.0
...,...,...,...,...
3064,NaN,NaN,81427.0,2.0
3065,NaN,NaN,81428.0,2.0
3067,NaN,NaN,81430.0,2.0
3069,NaN,NaN,81432.0,2.0


However, more in-depth explorations reveal that the IDs with excessive entries do not perfectly correspond across our two df's - for example:

- some IDs that are in **`df`** are not in **`df2`** and vice versa, and more importantly,
- some IDs that have duplicate entries in **`df`** do *not* have any duplicates in **`df2`**, adding more confusion to the matter.

💡 **Idea:** Let's just merge the df's and see what happens.

Some additional data preprocessing is necessary before merging. 

For **`df`**, the sum of all words that a child is capable of producing (i.e. the sum of **`value`**) is calculated. For each child ID, this calculated sum should correspond with the value of **`production`** in **`df2`**, as this variable represents a tally of how many of the 680 items the child in question is able to speak. When merging our datasets, we can use this as additional information (alongisde **`age`** and **`child_id`**) to help us match the children as accurately as possible, especially considering the inconsistencies found between the two df's.

Rows that do not match on all of these three variables are dropped.

In [37]:
dfx = df.groupby("child_id").agg({"value": "sum", "age": "first"}).reset_index()
dfx.rename(columns={"value": "production"}, inplace=True)
mergedf = dfx.merge(df2, on=["child_id", "age", "production"], how="inner")
mergedf = mergedf[["child_id", "age", "sex", "production"]]

In [38]:
mergedf

,child_id,age,sex,production
0,1,28,Female,497
1,2,22,Female,369
2,3,26,Female,190
3,4,27,Male,264
4,5,19,Female,159
...,...,...,...,...
4826,86611,22,Male,120
4827,86612,29,Male,676
4828,86613,22,Male,170
4829,86614,28,Female,313


Let's take a look at our new df:

In [39]:
print("\033[1mSummary:\033[0m")
print()
print("Total number of children recorded:", mergedf["child_id"].nunique())
print("Numer of rows:", mergedf.shape[0])

Summary:

Total number of children recorded: 4831
Numer of rows: 4831


No more duplicates! 🚀

The final step in our merge is to combine the information contained in **`mergedf`** with our main dataset **`df`**, and remove all entries that don't have a match.

In [40]:
final = df.merge(mergedf, on=["child_id", "age"], how="inner")

Check to see whether the number of IDs makes sense:

In [41]:
final["child_id"].nunique()

4831

It does! We now have 5233 different children, all typically developing, no duplicates, no NAs, and have complete information on how many words they can produce, their age, and their sex.

Finally, remove all the columns we don't need.

In [42]:
final = final.drop(columns=["downloaded", "data_id", "item_kind"])
final = final[["child_id", "age", "sex", "item_id", "item_definition", "value", "production"]]
final.head()

,child_id,age,sex,item_id,item_definition,value,production
0,1,28,Female,item_1,baa baa,1,497
1,1,28,Female,item_2,choo choo,0,497
2,1,28,Female,item_3,cockadoodledoo,0,497
3,1,28,Female,item_4,grrr,1,497
4,1,28,Female,item_5,meow,0,497


Save **`final`** as parquet.

In [90]:
final.to_parquet('preprocessed.parquet', index=False)